# Investigate impacts from the wording of Federal Reserve (Fed) meetings to financial market.

The objective of this project is to apply NLP on those texts published by FOMC to attempt to
quantify the communications in a systematic manner and find the relationship with Treasury
yield curve.

# Table of Contents


1. [Data Overview](#data-o)
    <br>1.1 [Summary](#sum)
    <br>1.2 [Sections](#t-sect)
    <br>1.3 [Crop Sections](#crop)

2. [Data Pre-processing](#data-p)
    <br>2.1 [Tokenization](#stemlem)

3. [Topic Modelling](#topic-m)
    <br>3.1 [Non-negative Matrix Factorization](#nmf)
    <br>3.2 [Latent Dirichlet Allocation](#lda)
 
4. [Sentiment Analysis](#sent-a)
    <br>4.1 [Dictionary Approach](#dict)
    <br>4.2 [Sentiment Extraction](#sent-e)
    
5. [Modeling](#model)
    <br>5.1 [Sentiment Correlations and Baseline Model](#base)
    <br>5.2 [BERT and CNN Model](#bert)







In [1]:
file_path_prefix = None

In [2]:
#if using colab
from google.colab import drive
drive.mount('/content/drive')
!pip install -r drive/My\ Drive\/fednlp/requirements.txt
file_path_prefix = './drive/My Drive/fednlp/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     |████████████████████████████████| 753.2MB 22kB/s 
     |████████████████████████████████| 6.6MB 20.3MB/s 
     |████████████████████████████████| 25.9MB 116kB/s 
     |████████████████████████████████| 9.9MB 60.7MB/s 
     |████████████████████████████████| 368kB 57.0MB/s 
     |████████████████████████████████| 1.4MB 53.7MB/s 
     |████████████████████████████████| 71kB 198kB/s 
     |████████████████████████████████| 778kB 6.6MB/s 
     |████████████████████████████████| 122kB 10.0MB/s 
     |████████████████████████████████| 2.1MB 13.1MB/s 
     |████████████████████████████████| 3.0MB 24.9MB/s 
     |████████████████████████████████| 890kB 44.5MB/s 
     |████████████████████████████████| 1.1MB 64.3MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434676 sha256=33346e77d57cb6e1d4dc423e1c2c91ec61bbd0166972b26bd4c6cb5a32329df9

In [3]:
import re
import os,sys
import calendar
import datetime
from datetime import timedelta
from collections import Counter
import pickle

import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

import nltk
from nltk.corpus import CategorizedPlaintextCorpusReader
from wordcloud import WordCloud
import spacy
from spacy.lang.en import English
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer


if file_path_prefix is None:
    file_path_prefix = '/Users/aaroncgw/Google Drive/fednlp/data/'

nltk.download('punkt')

!python -m spacy download en_core_web_md
import en_core_web_md
nlp = en_core_web_md.load()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     |████████████████████████████████| 50.8MB 2.4MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.3.1-cp36-none-any.whl size=50916643 sha256=4fbcf235e60825a743272d89f352677ab3f7a1c697efb9424547d4d5393da4c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-m8u4lgp2/wheels/6e/65/3a/34cdc26d4084d1d1f1e2ec9914964759ea17aa382c53a57d9f
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


# <a name="data-o"></a> Data Overview
Data was web scrapped from https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm.
The year 2004 was chosen as the start-year given the acceleration of release dates to 3 weeks, and improved clarity in explanations of committee’s decisions and views [(Danker,2005)](https://www.federalreserve.gov/pubs/bulletin/2005/spring05_fomc.pdf)

## <a name="sum"></a>Minutes Summary

In [ ]:
corpus_root = file_path_prefix + 'minutes'
data_m = CategorizedPlaintextCorpusReader(corpus_root, r'.*\.txt', cat_pattern=r'(\w+)/*')
data_fileids = data_m.fileids()

print('Total number of files: '+str(len(data_m.fileids())))
print('Number of paragraphs: '+str(len(data_m.paras())))
print('Number of sentences: '+str(len(data_m.sents())))
print('Number of words: '+str(len(data_m.words())))
print('\n'+'First file: '+ data_fileids[0])
print('Last file: '+ data_fileids[-1])

Total number of files: 130


In [ ]:
num_para_py = {}
num_word_py = {}

for y in range(2004,2021):
    files = data_m.fileids(str(y))
    files_size = len(files)
    num_para_py[y] = sum([len(data_m.paras(f))for f in files])/files_size
    num_word_py[y] = sum([len(data_m.words(f))for f in files])/files_size
        
para_words = pd.DataFrame([num_para_py,num_word_py],
                          index = ['Average number of paragraphs','Average number of words']).T

para_words = para_words.reset_index()

fig,ax = plt.subplots(2, 1, figsize=(10,10))
sns.barplot(x="index", y="Average number of paragraphs", data=para_words, ax=ax[0])
sns.barplot(x="index", y="Average number of words", data=para_words, ax=ax[1])

We can see an acceleration in the amount of paragraphs and words overtime.

## <a name="t-sect"></a> Sections
The minutes are split into 4 sections (see [Background on FOMC Meeting Minutes](https://www.federalreserve.gov/pubs/bulletin/2005/spring05_fomc.pdf)) 

1. Introduction
2. Economic and financial information
3. Participants’ views on developments 
4. Policy decisions

The first set of introductory paragraphs contain a list of attendees and procedural items, which are not of use to our analysis and hence need to be removed. The next problem to tackle is knowing when to start the minutes from.

In [ ]:
df_temp = pd.DataFrame()
df_year = pd.DataFrame()
for y in range(2004,2019):
    files = data_m.fileids(str(y))
    for f in files:
        word_para = [sum([len(s)for s in p]) for p in data_m.paras(f)]
        df_temp = pd.concat([df_temp,pd.Series(word_para,name=f)],axis=1)
    
    df_mean = df_temp.mean(axis=1)
    df_mean.name = y
    df_year= pd.concat([df_year,df_mean],axis=1)
    df_temp = df_temp.iloc[0:0]

window_size = 10

y1 = df_year.loc[:,[y for y in range(2004,2008)]].mean(axis=1).rolling(window_size).mean()
y2 = df_year.loc[:,[y for y in range(2009,2012)]].mean(axis=1).rolling(window_size).mean()
y3 = df_year.loc[:,[y for y in range(2012,2019)]].mean(axis=1).rolling(window_size).mean()

df_joint = pd.DataFrame([y1,y2,y3],index = ['2004-2007','2008-2011','2012-2018']).T


ax = df_joint.plot(figsize=(10,5))
ax.set_xlabel("Paragraph index")
ax.set_ylabel("Word count")

To see how the mintues are structured, we can plot a moving average of words per paragraph. The peaks and troughs of this moving average are indicative of new sections, where usually the start of the section includes a header or a very few words. 

For simplicity, I've identified three different types of structures that span from 2004-2007, 2008-2011 and 2012-2018. It is quite clear that the early minutes (2004-2007) began it's second section by paragraph 30-40, whilst those minutes later on began around paragraph 50 onwards.
  

## <a name="crop"></a>Crop Sections

Given that we know where sections roughly start, the easiest way to split our sections is to find the most common words. For our analysis, the minutes will start with phrases similar to 'Staff Review of the Economic Situation' or 'The information reviewed' and end on 'At the conclusion of this meeting' or 'The Committee voted to authorize'.

In [ ]:
corpus_root_cropped = file_path_prefix + 'minutes_cropped'
data_c = CategorizedPlaintextCorpusReader(corpus_root_cropped, r'.*\.txt', cat_pattern=r'(\w+)/*')

print('Total number of files: '+str(len(data_c.fileids())))
print('Number of paragraphs: '+str(len(data_c.paras())))
print('Number of sentences: '+str(len(data_c.sents())))
print('Number of words: '+str(len(data_c.words())))

Total number of paragraphs, sentences and words have been reduced by 30-60%.

## <a name="rates"></a>Treasury Rates

In [ ]:
with open(file_path_prefix + 'rates/rates_daily.pickle', 'rb') as handle:
    rates_daily = pickle.load(handle)
    
with open(file_path_prefix + 'rates/rates_monthly.pickle', 'rb') as handle:
    rates_monthly = pickle.load(handle)

In [ ]:
df_yc_slope_changes = pd.DataFrame((rates_monthly['BC_10YEAR'] - rates_monthly['BC_3MONTH']), columns=['slope_change']).diff().shift(-1)[:-1]
df_yc_slope_changes['steepen'] = df_yc_slope_changes['slope_change'] > 0
df_yc_slope_changes


# <a name="data-p"></a> Data Pre-processing



## <a name="stemlem"></a> Tokenization

To enhance topic modelling output, we can remove stop words and reduce inflectional forms of words back to its roots - using techniques such as stemming and lemmatization. 


In [ ]:
en = English()

def simple_tokenizer(doc, model=en):
    tokenized_docs = []
    parsed = model(doc)
    return([t.lemma_.lower() for t in parsed if (t.is_alpha)&(not t.like_url)&(not t.is_stop)])

In [ ]:
def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return next_month - datetime.timedelta(days=next_month.day)

In [ ]:
idx = []
paragraph_list = []
for f in data_c.fileids():
    year,month,day = re.search("(\d{4})(\d{2})(\d{2})",f).groups()
    paragraph_list_single_file = data_c.raw(f).split("\n\n")
    for i, paragraph in enumerate(paragraph_list_single_file):
        p_tokenized = simple_tokenizer(paragraph)
        if len(p_tokenized) != 0:
            idx.append((last_day_of_month(datetime.datetime(int(year), int(month), int(day))), i)) #Year,Month,Day,Paragraph number
            paragraph_list.append([p_tokenized,paragraph])
        
df_paragraphs = pd.DataFrame(data = paragraph_list,
                      index = pd.MultiIndex.from_tuples(idx), 
                      columns = ['tokenized_text','raw_text'])

print(len(df_paragraphs))
df_paragraphs.head()

In [ ]:
'''
idx = []
para_number = []
paragraph_list = []
for f in data_c.fileids():
    year,month,day = re.search("(\d{4})(\d{2})(\d{2})",f).groups()
    for i,p in enumerate(data_c.paras(f)):
        idx.append((last_day_of_month(datetime.datetime(int(year), int(month), int(day))), i)) #Year,Month,Day,Paragraph number
        flat_p = [item for sublist in p for item in sublist]
        paragraph = ' '.join(flat_p)
        p_tokenized = simple_tokenizer(paragraph)
        paragraph_list.append([p_tokenized,flat_p])

df_paragraphs = pd.DataFrame(data = paragraph_list,
                          index = pd.MultiIndex.from_tuples(idx), 
                          columns = ['tokenized_text','raw_text'])

df_paragraphs.head()'''

In [ ]:
#Most frequent words in corpus
tokenized_text = df_paragraphs['tokenized_text']
all_tokens = [item for sublist in tokenized_text for item in sublist]

wordcloud = WordCloud(collocations=False,
                      background_color='white',
                      mode='RGB',
                      width=1600, height=800).generate((" ").join(all_tokens))

fig = plt.figure(figsize=(16,8),dpi=200)
plt.axis('off')
plt.imshow(wordcloud, interpolation="bilinear")
plt.show()

# <a name="topic-m"></a>Topic Modelling 

Now that we have our clean text corpus, we can create a bag of words and pass it into a Latent Dirichlet Allocation model (LDA). This probabilistic model is ideal for our analysis as each paragraph is distributed amongst a set of topics, where topics themselves are defined by words that frequently appear with each other. 

<br>
In our previous example, the phrase "Labor market conditions continued to strengthen in recent months, with the unemployment rate declining further and payroll gains well above a pace consistent with maintaining a stable unemployment rate over time" is one that belongs in the labour market category. Additionally, one might say words like labour, payroll and unemployment are more likely to appear in the same document (or in this case paragraph). 

In [ ]:
def summarize_topics(model, feature_names, no_top_words):
    topics = pd.DataFrame()
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i]for i in topic.argsort()[:-no_top_words - 1:-1]]
        topics["Topic %d:" % (topic_idx)] = top_words
    return topics

num_topics = 6

## <a name="lda"></a>LDA

One of the most important inputs in the LDA model is the number of topics (n_components). Given that this is an unsupervised task, there is no best way to choose this input other than trial and error - which in this case results to n_components = 6. 

In [ ]:
count_vecs = CountVectorizer(tokenizer = lambda x: x, lowercase = False)
count_vecs_fitted = count_vecs.fit_transform(list(tokenized_text))

lda = LatentDirichletAllocation(n_components = num_topics, random_state=7).fit(count_vecs_fitted)
lda_t = lda.transform(count_vecs_fitted)

df_paragraphs['topic_weights'] = list(lda_t)

lda_topic_top_words = summarize_topics(lda,count_vecs.get_feature_names(),10)
lda_topic_top_words

The output above shows the top 10 words associated to each topic. Some topics are more distinguishable than others, where words such as inflation, price, energy are usually associated to the topic 'Inflation', whilst others are less so.

In [ ]:
#Dictionary with topics
topic_dict = {0 : 'Inflation',
              1 : 'Policy',
              2 : 'Investment', 
              3 : 'Financial Market',
              4 : 'Labor Market', 
              5 : 'Outlook'}
topic_dict

In [ ]:
def sum_prop(x):
    x_sum = x.sum(axis=0)
    return (x_sum/x_sum.sum())*100

df_topic_t = df_paragraphs.groupby(level=[0],sort=False)['topic_weights']\
                        .apply(lambda x: sum_prop(x)).apply(pd.Series)\
                        .rename(columns = topic_dict)

df_topic_m = df_topic_t.groupby(level=[0],sort=False).apply(lambda x: sum_prop(x))
topic_perc = df_topic_m.divide(df_topic_m.sum(axis=1), axis=0)

In [ ]:
df_topic_m.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
ax.stackplot(df_topic_m.index,  topic_perc["Labor Market"],  topic_perc["Policy"],  topic_perc["Financial Market"], 
              topic_perc["Inflation"],  topic_perc["Outlook"],  topic_perc["Investment"],
              labels=['Labor Market','Policy','Financial Market', 'Inflation', 'Outlook', 'Investment'])
ax.legend(loc='upper left')
ax.margins(0,0)
ax.set_title('Topic Percentage Over Time')

As expected the topic 'Financial markets' increased in proportion after 2008. To date, the largest topic is 'Policy' and the smallest is 'Labor Market'. 

# <a name="sent-a"></a> Sentiment Analysis 

Two sets of dictionaries were used in this section: 
1. Havard IV-4 Psychosociological 
2. Loughran and McDonald

The former is less tailored to financial statements, whilst Loughran and McDonald is adapted to include words from 10-K documents.

Given the lack of labelled data, a very simplistic approach is used to calculate sentiment tone: 

\begin{equation*}
Net \ Tone \  = \frac{\#Positive\ words - \#Negative\ words}
                    {\#Positive\ words + \#Negative\ words} \times \frac{1}{\#Total\ words}
\end{equation*}

Where <br>

Net tone > 0 points to a positive tone
<br>
Net tone < 0 points to a negative tone
<br>
Net tone = 0 points to neutral tone


##  <a name="dict"></a> Dictionary Approach 

In [ ]:
with open(file_path_prefix + 'sentiment/sentiment_pos_dict.pickle', 'rb') as handle:
    posDict = pickle.load(handle)
    
with open(file_path_prefix + 'sentiment/sentiment_neg_dict.pickle', 'rb') as handle:
    negDict = pickle.load(handle)


corpus_wcount = Counter([item for sublist in tokenized_text for item in sublist])
pos_inter = Counter({w:corpus_wcount[w] for w in posDict.intersection(corpus_wcount)})
neg_inter = Counter({w:corpus_wcount[w] for w in negDict.intersection(corpus_wcount)})

top_pos = pos_inter.most_common(10)
top_neg = neg_inter.most_common(10)

df_top_pos = pd.DataFrame(top_pos, columns=['token', 'count'])
df_top_neg = pd.DataFrame(top_neg, columns=['token', 'count'])

fig, ax = plt.subplots(1,2, figsize=(18,8))
sns.barplot(x="token", y="count", data=df_top_pos, ax=ax[0])
ax[0].set_title('top positive words')
sns.barplot(x="token", y="count", data=df_top_neg, label='top negative words', ax=ax[1])
ax[1].set_title('top negative words')

These are the top 10 positive and negative words that appear in the corpus. 

In [ ]:
print("Number of positive words in corpus: " +str(sum(pos_inter.values())))
print("Number of negative words in corpus: " +str(sum(neg_inter.values())))

## <a name="sent-e"></a> Sentiment Extraction

In [ ]:
def RetrieveScore(tokenized_para,positive,negative): 
    pos_sum = 0
    neg_sum = 0
    score = 0
    if len(tokenized_para) <8:
        return 0
    for word in tokenized_para: 
        if word in positive:
            pos_sum +=1
        elif word in negative:
            neg_sum +=1
    try:
        score = ((pos_sum-neg_sum)/(pos_sum+neg_sum))*(1/len(tokenized_para)) #should this be 
    except ZeroDivisionError:
        score = 0
    return score

df_paragraphs['sentiment_score'] = df_paragraphs.apply(lambda x: RetrieveScore(x['tokenized_text'],posDict,negDict),axis=1)
df_paragraphs


In [ ]:
df_sentiment = (df_paragraphs['sentiment_score']*df_paragraphs['topic_weights']).apply(pd.Series).rename(columns = topic_dict)
df_sentiment = pd.concat([df_sentiment.sum(axis=1).rename('Total'),df_sentiment],axis=1)
df_sentiment_m = df_sentiment.groupby(level = [0],sort=False).apply(lambda x: x.sum())
df_sentiment_m.plot(figsize=(18,8))

The downside of using such a simplistic approach in calculating sentiment is that you can produce some anomalies, in particular for the 'inflation' topic. The word 'inflation' is negative but context is very important when classifying sentiment - in this case the sentence 'Inflation slowed down' should be a positive phrase but is classified as negative. Nevertheless, we can see see a joint dip amongst all topics in 2008, relating to the financial crisis. 

In [ ]:
df_paragraphs.head()

In [ ]:
timestamps = df_paragraphs.index.map(lambda x: str(x[0]))
paragragh_numbers = df_paragraphs.index.map(lambda x: str(x[1]))
#unstacked_idx = [last_day_of_month(pd.to_datetime(i, format="%Y%b")) for i in idx]
df_paragraphs['paragragh_number'] = paragragh_numbers
df_paragraphs.index = timestamps

In [ ]:
df_paragraphs = df_paragraphs.join(df_yc_slope_changes)
df_paragraphs

## <a name="rates"></a> Sentiment Correlations

In [ ]:
combined_df = df_sentiment_m.join(df_yc_slope_changes.slope_change)
combined_df.corr()

In [ ]:
print(scipy.stats.pearsonr(combined_df['Total'], combined_df['slope_change']))
print(scipy.stats.pearsonr(combined_df['Labor Market'], combined_df['slope_change']))
print(scipy.stats.pearsonr(combined_df['Policy'], combined_df['slope_change']))
print(scipy.stats.pearsonr(combined_df['Financial Market'], combined_df['slope_change']))
print(scipy.stats.pearsonr(combined_df['Inflation'], combined_df['slope_change']))
print(scipy.stats.pearsonr(combined_df['Outlook'], combined_df['slope_change']))
print(scipy.stats.pearsonr(combined_df['Investment'], combined_df['slope_change']))

In [ ]:
fig, ax_left = plt.subplots(figsize=(20,10))
ax_right = ax_left.twinx()

ax_left.plot(combined_df['Financial Market'], color='blue', label='Financial Market')
ax_right.plot(combined_df['slope_change'], color='red', label='yield curve slope change')
plt.legend()

In [ ]:
plt.figure()
f, axes = plt.subplots(2, 1, figsize=(20,10))
axes[0].plot(combined_df.index, combined_df['Financial Market'])
axes[0].set_ylabel('Financial Market')

axes[1].plot(combined_df.index, combined_df['slope_change'])
axes[1].set_ylabel('slope_change')

# <a name="model"></a>Modeling

In [ ]:
x = df_paragraphs['raw_text']
y = df_paragraphs['steepen']

indices = list(range(len(y)))
minutes_train, minutes_val, is_steepen_train, is_steepen_val, indices_train, indices_val = train_test_split(x, y, indices, test_size=0.3, random_state=7)
print(len(minutes_val))
print(len(is_steepen_val))

## <a name="base"></a>Baseline Model

In [ ]:
pipe = Pipeline([('vectorizer', TfidfVectorizer(tokenizer=simple_tokenizer)), ('svc', LinearSVC())])
pipe.fit(minutes_train, is_steepen_train)
print("Tfidf Train accuracy: ", pipe.score(minutes_train, is_steepen_train))
print("Tfidf Test accuracy: ", pipe.score(minutes_val, is_steepen_val))

In [ ]:
pipe = Pipeline([('vectorizer', TfidfVectorizer(tokenizer=simple_tokenizer)), ('topics', NMF(n_components=6, random_state=7)), ('svc', LinearSVC())])
pipe.fit(minutes_train, is_steepen_train)
print("NMF Train accuracy: ", pipe.score(minutes_train, is_steepen_train))
print("NMF Test accuracy: ", pipe.score(minutes_val, is_steepen_val))


In [ ]:
'''
cv = CountVectorizer(tokenizer=simple_tokenizer)
count_vectors_train = cv.fit_transform(minutes_train)
count_vectors_val = cv.transform(minutes_val)

lda = LatentDirichletAllocation(n_components=6)
lda_train_vecs = lda.fit_transform(count_vectors_train)
lda_test_vecs = lda.transform(count_vectors_val)
'''

#combine both LDA and sentiment scores
lda_sen = np.hstack([np.vstack(df_paragraphs['topic_weights']), np.vstack(df_paragraphs['sentiment_score'])])

baseline_train_vecs = lda_sen[indices_train]
baseline_val_vecs = lda_sen[indices_val]

svc = LinearSVC()
svc.fit(baseline_train_vecs, is_steepen_train)
train_preds = svc.predict(baseline_train_vecs)
test_preds = svc.predict(baseline_val_vecs)

print('Baseline Model Train accuracy:', accuracy_score(is_steepen_train, train_preds))
print('Baseline Model Test accuracy:', accuracy_score(is_steepen_val, test_preds))

## <a name="bert"></a>GloVe and BERT Embeddings 

In [ ]:
vocab = set([value for valueList in tokenized_text.values for value in valueList ])
vocab.add('_UNK')
vocab.add('_PAD')

print("Size of vocab:", len(vocab))

glove_dict = dict()
for k in vocab:
    glove_dict[k] = nlp(k).vector

In [ ]:
glove_vectors = np.empty(shape=(len(df_paragraphs), 300))
for row, text in enumerate(df_paragraphs['tokenized_text']):
    glove_vectors_arrays = [glove_dict[word] for word in text]
    glove_vectors[row] = np.mean(glove_vectors_arrays, axis=0)

In [ ]:
#combine both GloVe and sentiment scores
glove_sen = np.hstack([glove_vectors, np.vstack(df_paragraphs['sentiment_score'])])

glove_train_vecs = glove_sen[indices_train]
glove_val_vecs = glove_sen[indices_val]

svc = LinearSVC()
svc.fit(glove_train_vecs, is_steepen_train)
train_preds = svc.predict(glove_train_vecs)
test_preds = svc.predict(glove_val_vecs)

print('Baseline Model Train accuracy:', accuracy_score(is_steepen_train, train_preds))
print('Baseline Model Test accuracy:', accuracy_score(is_steepen_val, test_preds))

In [ ]:
# basic pretrained model (case-insensitive)
MODEL_NAME = 'distilbert-base-uncased'
# Load pre-trained model
model = DistilBertModel.from_pretrained(MODEL_NAME)
# Load pre-trained model tokenizer (vocabulary)
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
samples = np.random.choice(df_paragraphs['raw_text'], size=5)
samples.shape

In [ ]:

tokens = tokenizer.batch_encode_plus(df_paragraphs['raw_text'],
    pad_to_max_length=True, # this implements padding for different length docs
    return_tensors="pt", # returning as pytorch tensors
    max_length=512,
    truncation=True) # this BERT model has a max sequence length of 512 tokens

outputs = model(**tokens)
outputs[0][:,0].detach().numpy().shape